<a href="https://colab.research.google.com/github/ckh0103/data_anal/blob/main/ch03_reg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 필요한 파일 다운 받기
import gdown
gdown.download("https://bit.ly/3RhoNho", "ns_202104.csv", quiet=False)

Downloading...
From: https://bit.ly/3RhoNho
To: /content/ns_202104.csv
100%|██████████| 57.6M/57.6M [00:00<00:00, 214MB/s]


'ns_202104.csv'

In [2]:
# pandas로 데이터 일어오기
import pandas as pd
ns_df = pd.read_csv("ns_202104.csv", low_memory=False)
# 13 의미없는 열 제거
ns_book = ns_df.loc[:, "번호":"등록일자"]

In [3]:

selected_rows = ns_df["출판사"] == "한빛미디어"
ns_book2 = ns_book[selected_rows]
count_df = ns_book[["도서명", "저자", "ISBN", "권", "대출건수"]]
group_df = count_df.groupby(by=["도서명", "저자", "ISBN", "권"], dropna=False)
loan_count = group_df.sum()
# 도서 대여가 중복이 되는 데이터
# 처음 데이터는 중복이 되지 않았다. - False, 2번 데이터부터는 중복이 되었다. - True
dup_rows = ns_book.duplicated(subset=["도서명", "저자", "ISBN", "권"])
# 중복이 안되는 데이터
unique_rows = ~dup_rows
ns_book3 = ns_book[unique_rows].copy()
ns_book3.set_index(["도서명", "저자", "ISBN", "권"], inplace=True)
ns_book3.update(loan_count)
ns_book4 = ns_book3.reset_index()
ns_book4.head()



,도서명,저자,ISBN,권,번호,출판사,발행년도,세트 ISBN,부가기호,주제분류번호,도서권수,대출건수,등록일자
0,인공지능과 흙,김동훈 지음,9788937444319,NaN,1,민음사,2021,NaN,NaN,NaN,1,0,2021-03-19
1,가짜 행복 권하는 사회,김태형 지음,9791190123969,NaN,2,갈매나무,2021,NaN,NaN,NaN,1,0,2021-03-19
2,나도 한 문장 잘 쓰면 바랄 게 없겠네,김선영 지음,9788968332982,NaN,3,블랙피쉬,2021,NaN,NaN,NaN,1,0,2021-03-19
3,예루살렘 해변,"이도 게펜 지음, 임재희 옮김",9788970759906,NaN,4,문학세계사,2021,NaN,NaN,NaN,1,0,2021-03-19
4,김성곤의 중국한시기행 : 장강·황하 편,김성곤 지음,9788934990833,NaN,5,김영사,2021,NaN,NaN,NaN,1,0,2021-03-19


In [4]:

# 정규 표현식을 이용한 데이터 수정
# replace({"키:항목":{키 찾는데이타:바꿀데이터}})
# 찾는 데이터를 정규 표현식 활용한 바꾸기
# 일반적인 replace
# ns_book4.replace({"발행년도":{"2021":"21"}})[100:102]
# 정규 표현식을 이용한 replce : 한번에 yyyy -> yy 형식으로 바꾼다. r"문자열" : 문자열이 정규 표현식
# 정규 표현식의 \d -> 숫자 데이터, () -> 그룹으로 만듬.
# r"\1" - 첫번재 그룹의 데이터
# ns_book4.replace({"발행년도":{r"\d\d(\d\d)": r"\1"}}, regex=True)[100:102]
# 정규표현식 문자/문자열의 반복 : 바로 뒤에 {} - {2} - 2번 반복되어야 한다. {2,4} - 2번부터 4번 반복한다.
ns_book4.replace({"발행년도":{r"\d{2}(\d{2})": r"\1"}}, regex=True)[100:102]



,도서명,저자,ISBN,권,번호,출판사,발행년도,세트 ISBN,부가기호,주제분류번호,도서권수,대출건수,등록일자
100,No라고 말할 줄 아는 남편과 아내 - 개정판,"헨리 클라우드, 존 타운센드 (지은이), 김진웅 (옮긴이)",9788958743019,NaN,101,좋은씨앗,18,NaN,NaN,234.9,1,1,2021-03-15
101,"D2C 레볼루션 - 스타트업부터 글로벌 기업까지, 마켓 체인저의 필수 전략","로런스 인그래시아 (지은이), 안기순 (옮긴이)",9788960518483,NaN,102,부키,21,NaN,NaN,325.1,1,0,2021-03-15


In [ ]:

# 저장 항목에서 (지은이), (옮긴이) 제거해 보자
# () - 그룹으로 처리,  - 순수 문자열 ()로 처리
# . - 모든 문자, \s - 공백문자
# (.*) - 지은이 앞에 사람이름 : 첫번째 그룹 - \1, (.*) - 옮긴이 앞에 사람이름 + 앞에 "," + "blk" : 두번째 그룸 - \2
ns_book4.replace({"저자":{r"(.*)지은이(.*)\s옮긴이": r"\1\2"}}, regex=True)[100:102]
# ns_book4.replace({"저자":{r"옮긴이": ""}}, regex=True, inplace=True)
# ns_book4.head()
# ns_book4[100:102]

,도서명,저자,ISBN,권,번호,출판사,발행년도,세트 ISBN,부가기호,주제분류번호,도서권수,대출건수,등록일자
100,No라고 말할 줄 아는 남편과 아내 - 개정판,"헨리 클라우드, 존 타운센드 (지은이), 김진웅 (옮긴이)",9788958743019,NaN,101,좋은씨앗,2018,NaN,NaN,234.9,1,1,2021-03-15
101,"D2C 레볼루션 - 스타트업부터 글로벌 기업까지, 마켓 체인저의 필수 전략","로런스 인그래시아 (지은이), 안기순 (옮긴이)",9788960518483,NaN,102,부키,2021,NaN,NaN,325.1,1,0,2021-03-15


In [ ]:

ns_book4.replace({"저자":{r"(.*)\s지음(.*)\s옮김": r"\1\2"}}, regex=True).replace({"저자":{r"(.*)\s지음": r"\1"}}, regex=True)[:4]



,도서명,저자,ISBN,권,번호,출판사,발행년도,세트 ISBN,부가기호,주제분류번호,도서권수,대출건수,등록일자
0,인공지능과 흙,김동훈,9788937444319,NaN,1,민음사,2021,NaN,NaN,NaN,1,0,2021-03-19
1,가짜 행복 권하는 사회,김태형,9791190123969,NaN,2,갈매나무,2021,NaN,NaN,NaN,1,0,2021-03-19
2,나도 한 문장 잘 쓰면 바랄 게 없겠네,김선영,9788968332982,NaN,3,블랙피쉬,2021,NaN,NaN,NaN,1,0,2021-03-19
3,예루살렘 해변,"이도 게펜, 임재희",9788970759906,NaN,4,문학세계사,2021,NaN,NaN,NaN,1,0,2021-03-19


In [ ]:

# 발행년도를 숫자로 변경해보자 -> 숫자가 아닌 데이터가 포함되어 있다.
ns_book4.astype({"발행년도":"int64"})

ValueError: invalid literal for int() with base 10: '1988.': Error while type casting for column '발행년도'

In [ ]:
ns_book4['발행년도'].str.contains("1988").sum()

407

In [5]:
# contains(str = 정규표현식 | r"" -> r을 생략해도 된다.)
# 발행년도에서 숫자로 변경 불가능한 데이터 확인
# \D - 숫자가 아닌 것, \d - 숫자 인것

invalid_number = ns_book4["발행년도"].str.contains("\D", na = True)
print((invalid_number).sum())
ns_book4[invalid_number]

1777


,도서명,저자,ISBN,권,번호,출판사,발행년도,세트 ISBN,부가기호,주제분류번호,도서권수,대출건수,등록일자
19138,단국강토,홍태수 저,9788974420031,NaN,19565,매일경제신문사,1988.,NaN,NaN,511.1,1,1,2019-12-19
19227,삼성의 역사,송부웅 撰,9788985464369,NaN,19736,삼양,단기4334[2001],NaN,0,911.02,1,1,2019-12-06
26097,배고픈 애벌레,에릭 칼 글·그림 ;이희재 옮김,9788959514083,NaN,26812,더큰컴퍼니,[2019],NaN,NaN,843,1,0,2019-08-12
29817,(The) Sopranos sessions,"Matt Zoller Seitz,$eAlan Sepinwall,$eLaura Lip...",9781419734946,NaN,30586,Harry N Abrams Inc,2019.,NaN,NaN,326.76,1,0,2019-06-13
29940,다음엔 너야,에른스트 얀들 글;노르만 융에 그림;박상순 옮김,9788949110646,NaN,30709,비룡소,2018.,9788949110004,7,853,1,9,2019-06-04
...,...,...,...,...,...,...,...,...,...,...,...,...,...
384573,法과 環境,韓國法學敎授會 編,9788935200078,NaN,401655,靑林出版,1977.,NaN,NaN,517.4,1,0,1970-01-01
384581,날마다 눈을 닦으며,박명용 저,9788970710198,NaN,401663,대장간,1992.,NaN,NaN,811.6,1,0,1970-01-01
384582,다시 故鄕에서,홍신선 지음,9788971907009,NaN,401665,동학사,1990.,NaN,NaN,811.6,1,0,1970-01-01
384583,익명의 편지,강계순 지음,9788930301688,NaN,401666,집문당,1990.,NaN,NaN,811.6,1,0,1970-01-01


In [6]:
# 단기4334[2001]	데이터로 정규표현식 r'.*(\d{4}).*' 처리를 하면 2001 이 나온다.
# 탐욕적 검색으로 마지막 매칭까지 찾아가서 현재 위치 마지막 숫자인 2001이 된다.
ns_book5 = ns_book4.replace({'발행년도':r'.*(\d{4}).*'}, r'\1', regex=True)
ns_book5[invalid_number].head()


,도서명,저자,ISBN,권,번호,출판사,발행년도,세트 ISBN,부가기호,주제분류번호,도서권수,대출건수,등록일자
19138,단국강토,홍태수 저,9788974420031,NaN,19565,매일경제신문사,1988,NaN,NaN,511.1,1,1,2019-12-19
19227,삼성의 역사,송부웅 撰,9788985464369,NaN,19736,삼양,2001,NaN,0,911.02,1,1,2019-12-06
26097,배고픈 애벌레,에릭 칼 글·그림 ;이희재 옮김,9788959514083,NaN,26812,더큰컴퍼니,2019,NaN,NaN,843,1,0,2019-08-12
29817,(The) Sopranos sessions,"Matt Zoller Seitz,$eAlan Sepinwall,$eLaura Lip...",9781419734946,NaN,30586,Harry N Abrams Inc,2019,NaN,NaN,326.76,1,0,2019-06-13
29940,다음엔 너야,에른스트 얀들 글;노르만 융에 그림;박상순 옮김,9788949110646,NaN,30709,비룡소,2018,9788949110004,7,853,1,9,2019-06-04


In [7]:
# 발행년도가 숫자가 아닌 데이터 확인

unknown_year = ns_book5['발행년도'].str.contains('\D', na = True)
print(unknown_year.sum())
ns_book5[unknown_year].head()

67


,도서명,저자,ISBN,권,번호,출판사,발행년도,세트 ISBN,부가기호,주제분류번호,도서권수,대출건수,등록일자
30838,본격 한중일 세계사 5,굽시니스트 지음,9791190065092,NaN,31616,위즈덤하우스,NaN,NaN,NaN,NaN,0,0,2019-05-28
39130,정책금융의 현황과 발전과제,정책금융연구회,9788992784108,NaN,40141,한국산업은행,NaN,NaN,NaN,327.1,1,0,2019-01-22
39256,서울지역 유적 발굴조사 총서 3,서울역사박물관,9791186324714,NaN,40268,서울역사박물관,NaN,9791186324431,NaN,NaN,0,0,2019-01-22
76836,흰머리 큰줄기,한호진 지음,9788973010769,NaN,81202,秀文出版社,[발행년불명],NaN,0,699.1,1,1,2016-11-10
150543,(속) 경제학사,박장환 지음,9788994339207,NaN,160436,NaN,[20--],NaN,1,320.9,1,2,2012-11-19


In [10]:
# 숫자가 아닌 데이터 처리
# 숫자가 아닌 데이터에 '-1' 로 세팅 그리고 int 로 변경 다시 저장해서 원본 데이터 변경

ns_book5.loc[unknown_year,'발행년도'] = "-1"
ns_book5 = ns_book5.astype({'발행년도': 'int64'})

# gt() > 4000 -> 단기 4334 데이터만 존재해서 숫자 데이터로 잡혔다.
ns_book5['발행년도'].gt(4000).sum()

np.int64(131)

In [11]:
# 4000 보다 큰 경우 2333을 빼서 서기로 바뀐다. 처리해 줘야한다.
(ns_book5['발행년도'] > 4000).sum()

np.int64(131)

In [12]:

dangun_yy_rows = ns_book5['발행년도'].gt(4000)

# 단기를 서기로 변환 후 다시 넣는다.
ns_book5.loc[dangun_yy_rows, '발행년도'] = ns_book5.loc[dangun_yy_rows, '발행년도'] - 2333

dangun_year = ns_book5['발행년도'].gt(4000)
print(dangun_year.sum())

ns_book5[dangun_year].head(2)

13


,도서명,저자,ISBN,권,번호,출판사,발행년도,세트 ISBN,부가기호,주제분류번호,도서권수,대출건수,등록일자
222858,Modern art,[computer file],9780717233243,NaN,234268,GROLIER,7611,NaN,NaN,609.205,1,1,2009-05-07
270269,현대 영어학=Linguistic theory in modern english,이재영,9788986705072,NaN,282852,열림기획,7634,NaN,1,740.1,1,6,2007-04-14


In [13]:
ns_book5.loc[dangun_year, '발행년도'] = -1

In [14]:
old_books = ns_book5['발행년도'].gt(0) & ns_book5['발행년도'].lt(1900)
ns_book5[old_books]

,도서명,저자,ISBN,권,번호,출판사,발행년도,세트 ISBN,부가기호,주제분류번호,도서권수,대출건수,등록일자
61149,"고흐 씨, 시 읽어 줄까요 :내 마음을 알아주는 시와 그림의 만남",이운진 지음,9788958284277,NaN,64093,사계절,176,NaN,0,811.7,1,6,2017-10-27
70566,정신 병리학의 문제들,지은이: 지그문트 프로이트 ;옮긴이: 황보석,9788932905181,10,74347,열린책들,151,9788932905082,9,185.5,1,4,2017-04-26
79550,(최근 7년간) 중요 민법판례 :[2009년 1월~2016년 6월 중요 판례],이광섭 편저,9788962898651,NaN,84164,법학사,163,NaN,9,365,1,14,2016-09-27
147950,(한·중·일) 밥상 문화 :대표음식으로 본 3국 문화비교,김경은 지음,9788958643012,NaN,157759,이가서,132,NaN,0,381.75,1,30,2013-02-19
194576,책으로 뒤덮인 집의 비밀,N.E. 보드 지음 ;피터 퍼거슨 그림 ;김지현 옮김,9788992844413,NaN,205407,개암나무,1015,NaN,7,843,1,15,2010-08-18
287252,(밝혀질)우리歷史,吳在城 著,9788986892130,NaN,300283,黎民族史硏究會,1607,NaN,0,911,1,5,2006-07-06


In [15]:
ns_book5.loc[old_books, '발행년도'] = -1
ns_book5['발행년도'].eq(-1).sum()

np.int64(86)

In [16]:

na_rows = ns_book5['도서명'].isna() | ns_book5['저자'].isna() | ns_book5['출판사'].isna() | ns_book5['발행년도'].eq(-1)
print(na_rows.sum())
ns_book5[na_rows].head(2)

5268


,도서명,저자,ISBN,권,번호,출판사,발행년도,세트 ISBN,부가기호,주제분류번호,도서권수,대출건수,등록일자
78,아산 정주영 레거시,김화진,9788952129529,NaN,79,NaN,2021,NaN,NaN,325,1,1,2021-03-15
265,골목의 시간을 그리다,정명섭.김효찬 지음,9791191266054,NaN,278,NaN,2021,NaN,NaN,NaN,1,0,2021-03-12


In [17]:
# HTML 코드를 가져오기 위한 라이브러리
import requests

# HTML 분석 라이브러리
from bs4 import BeautifulSoup

In [18]:
# 제목을 가져오자

def get_book_title(isbn):
  url = "https://www.yes24.com/product/search?domain=BOOK&query={}"

  r = requests.get(url.format(isbn))

  #soup - html 을 분석하기 위해 BeautifulSoup로 만들어 놓은 트리구조의 데이터
  soup = BeautifulSoup(r.text, 'html.parser')

  title = soup.find('a', attrs = {'class':'gd_name'}).get_text()
  return title

In [19]:
# 위의 코드를 전부 실행 해놔야 한다.
# 함수 호출 해서 책 제목을 가져오기

get_book_title(9791191266054)

'골목의 시간을 그리다'

In [20]:
# 비어 있는 데이터를 위한 웹 스크래핑을 이용한 데이터 가져오기

# findall() 함수를 위한 라이브러리
import re

# 데이터프레임의 한 줄을 받아서 처리하는 함수
# na_rows : 한 행씩 넘겨서 처리한다.
def get_book_info(row):

  # 가져와야할 데이터 선언 (비어 있는 경우만 처리하자)
  # pd.isna()
  title  = row['도서명']
  author = row['저자']
  pub    = row['출판사']
  year   = row['발행년도']

  url = "https://www.yes24.com/product/search?domain=BOOK&query={}"

  r = requests.get(url.format(row['ISBN']))

  #soup - html 을 분석하기 위해 BeautifulSoup로 만들어 놓은 트리구조의 데이터
  soup = BeautifulSoup(r.text, 'html.parser')


  #제목 가져오기
  try:
    if pd.isna(title):  # 제목이 비어있으면 실행한다.
      title = soup.find('a', attrs={'class':'gd_name'}).get_text()
  except AttributeError:
    pass

  try:
    if pd.isna(author):
      authors = soup.find('span', attrs={'class':'info_auth'}).find_all('a')
      author_list = [auth.get_text() for auth in authors]
      author = ','.join(author_list)
  except AttributeError:
    pass

  try:
    if pd.isna(pub):
      pub = soup.find('span', attrs={'class':'info_pub'}).find('a').get_text()
  except AttributeError:
    pass

  try:
    if year == -1:
      year_str = soup.find('span',attrs={'class':'info_date'}).get_text()
      year = re.findall(r'\d{4}', year_str)[0]
  except AttributeError:
    pass

  return title, author, pub, year



In [22]:
# 원래 수정해야할 개수는 많다. 시간이 오래 걸리고 로봇으로 인지되어 실행되지 않을 수도 있다.
# 그래서 2개의 데이터만 처리
updated_sample = ns_book5[na_rows].head(2).apply(get_book_info, axis=1, result_type='expand')
updated_sample

,0,1,2,3
78,아산 정주영 레거시,김화진,서울대학교출판문화원,2021
265,골목의 시간을 그리다,정명섭.김효찬 지음,초록비책공방,2021


In [23]:
updated_sample.columns = ["도서명", "저자", "출판사", "발행년도"]
updated_sample

,도서명,저자,출판사,발행년도
78,아산 정주영 레거시,김화진,서울대학교출판문화원,2021
265,골목의 시간을 그리다,정명섭.김효찬 지음,초록비책공방,2021


In [ ]:
# 데이터가 비어 있는 줄을 대상은 위에 5개만 처리
"""
for idx, row in ns_book5[na_rows].head(5).iterrows():

  # 웹에서 정보 가져오기
  title, author, pub, year = get_book_info(row)

  # 웬래 데이터 프레임에 직접 대입 - 가져온 데이터를 넣자.
  ns_book5.at[idx, '도서명'] = title
  ns_book5.at[idx, '저자'] = author
  ns_book5.at[idx, '출판사'] = pub
  ns_book5.at[idx, '발행년도'] = year
"""

In [ ]:
# 바꿔진 데이터 확인 - yes24에 없는 도서는 데이터가 비어있다.
#ns_book5[na_rows].head(5)

,도서명,저자,ISBN,권,번호,출판사,발행년도,세트 ISBN,부가기호,주제분류번호,도서권수,대출건수,등록일자
78,아산 정주영 레거시,김화진,9788952129529,NaN,79,서울대학교출판문화원,2021,NaN,NaN,325,1,1,2021-03-15
265,골목의 시간을 그리다,정명섭.김효찬 지음,9791191266054,NaN,278,초록비책공방,2021,NaN,NaN,NaN,1,0,2021-03-12
354,한국인의 맛,정명섭 지음,9791155401804,NaN,367,추수밭,2021,NaN,NaN,NaN,1,0,2021-03-12
539,"한성부, 달 밝은 밤에",김이삭 지음,9791163161479,NaN,552,고즈넉이엔티,2021,NaN,NaN,NaN,1,0,2021-03-12
607,100일 완성 마그마 러시아어 중고급 단어장,러포자 구제 연구소 외 지음,9788974829193,NaN,620,문예림,2021,NaN,NaN,NaN,1,0,2021-03-12


In [24]:
# 5268 건 전부 다 처리를 해야 한다.
# 미리 준비된 실행한 데이터를 다운받아 사용
gdown.download("https://bit.ly/3UJZiHw", "ns_book5_update.csv", quiet=False)
ns_book5_update = pd.read_csv("ns_book5_update.csv", low_memory=False, index_col=0)
ns_book5_update.head()

Downloading...
From: https://bit.ly/3UJZiHw
To: /content/ns_book5_update.csv
100%|██████████| 406k/406k [00:00<00:00, 52.1MB/s]


,도서명,저자,출판사,발행년도
78,아산 정주영 레거시,김화진,서울대학교출판문화원,2021
265,골목의 시간을 그리다,정명섭.김효찬 지음,초록비책공방,2021
354,한국인의 맛,정명섭 지음,추수밭,2021
539,"한성부, 달 밝은 밤에",김이삭 지음,고즈넉이엔티,2021
607,100일 완성 마그마 러시아어 중고급 단어장,러포자 구제 연구소 외 지음,문예림,2021


In [25]:
# ns_book5를 가지고 없다고한 데이터를 웹스크래핑한 데이터 ns_book5_update 자료로 업데이트한다.
ns_book5.update(ns_book5_update)

# 데이터가 비어있는 행
# 처리문이 다음 줄에 계속 이어지는 경우 \를 붙인다.
na_rows = ns_book5['도서명'].isna() | ns_book5['저자'].isna() | ns_book5['출판사'].isna() | ns_book5['발행년도'].eq(-1)

print(na_rows.sum())



4615


In [29]:
# 데이터 채우기 후에도 비어 있는 데이터는 제거한다.
# 도서명, 저자, 출판사에 데이터가 NaN인 것을 제거한다.
ns_book6 = ns_book5.dropna(subset=['도서명','저자','출판사'])
# 발행년도가 -1인 데이터 제거 : -1이 아닌 데이터를 골라서 다시 저장
ns_book6 = ns_book6[ns_book6['발행년도'] != -1]
#ns_book6 = ns_book6[~ns_book6['발행년도'].eq(-1)]

ns_book6.head()

,도서명,저자,ISBN,권,번호,출판사,발행년도,세트 ISBN,부가기호,주제분류번호,도서권수,대출건수,등록일자
0,인공지능과 흙,김동훈 지음,9788937444319,NaN,1,민음사,2021,NaN,NaN,NaN,1,0,2021-03-19
1,가짜 행복 권하는 사회,김태형 지음,9791190123969,NaN,2,갈매나무,2021,NaN,NaN,NaN,1,0,2021-03-19
2,나도 한 문장 잘 쓰면 바랄 게 없겠네,김선영 지음,9788968332982,NaN,3,블랙피쉬,2021,NaN,NaN,NaN,1,0,2021-03-19
3,예루살렘 해변,"이도 게펜 지음, 임재희 옮김",9788970759906,NaN,4,문학세계사,2021,NaN,NaN,NaN,1,0,2021-03-19
4,김성곤의 중국한시기행 : 장강·황하 편,김성곤 지음,9788934990833,NaN,5,김영사,2021,NaN,NaN,NaN,1,0,2021-03-19


In [30]:
ns_book6.to_csv('ns_book6.csv', index=False)